###  Agent for CV and Job Description Alignment

**Task :** To Provide user with Job Matching and Improvement Recommendation

#### 流程：
1. **第一階段（工作內容與條件案由）**：請 LLM 概述工作內容與條件(教育背景,技能,經驗需求)
2. **第二階段（履歷案由）**：請 LLM 概述履歷(教育背景,技能,經驗需求,成就)
3. **第三階段（履歷與工作比較）**：請 LLM 比較第一二階段的結果(機能相同於差異,經驗比較,整體比賽評分)
4. **第四階段（履歷修改推薦）**：請 LLM 使用第一二階段的結果來推薦履歷修改也包含了面試準備推薦與個人發展推薦



#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [ ]:
# Groq
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: groq
    Found existing installation: groq 0.26.0
    Uninstalling groq-0.26.0:
      Successfully uninstalled groq-0.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
provider_job_smmarizer = "groq"
model_job_smmarizer="llama3-70b-8192"

provider_cv_smmarizer = "groq"
model_cv_smmarizer="llama3-70b-8192"

provider_comparer = "groq"
model_comparer = "llama3-70b-8192"

provider_recommender  = "groq"
model_recommender  = "llama3-70b-8192"


In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [ ]:
system_job_summarizer='''
You are a professional job requirement analyzer. Extract and organize the key information from the provided job description into the following structured format:

1. **Role Overview**: Summarize the position in 1-2 sentences
2. **Required Hard Skills**: List technical skills, tools, platforms, and qualifications in bullet points
3. **Required Soft Skills**: List interpersonal and professional attributes in bullet points
4. **Experience Requirements**: List years of experience and specific background needed
5. **Education/Certification Requirements**: List degrees or certifications required
6. **Key Responsibilities**: List 3-5 main job duties

Present this information concisely from a recruiter's perspective, highlighting what would make a candidate qualified for this position.
'''

In [ ]:
system_cv_summarizer="""
You are a professional CV analyzer. Extract and organize the key information from the provided CV into the following structured format:

1. **Professional Summary**: Summarize the candidate's background in 1-2 sentences
2. **Hard Skills**: List technical skills, tools, platforms, and qualifications in bullet points
3. **Soft Skills**: List interpersonal and professional attributes in bullet points
4. **Experience Summary**: Summarize years of experience and specific background
5. **Education/Certifications**: List degrees or certifications held
6. **Notable Achievements**: List 2-3 standout accomplishments

Present this information concisely from a recruiter's perspective, highlighting what makes this candidate potentially valuable to employers.
"""

In [ ]:
system_comparer="""
You are a professional job-candidate matching specialist. Compare the provided job requirement summary and CV summary to evaluate compatibility.

Please analyze:
1. **Skills Match**: Identify overlapping hard and soft skills (strong matches)
2. **Skills Gap**: Identify required skills missing from the CV (weak matches)
3. **Experience Alignment**: Evaluate if the candidate's experience level meets requirements
4. **Overall Match Rating**: Provide a percentage score (0-100%) of how well the candidate matches the job
5. **Key Strengths**: List 2-3 aspects that make the candidate particularly suitable
6. **Key Concerns**: List 2-3 potential shortcomings for this role

Present this as an objective assessment a hiring manager would use to decide whether to interview this candidate.
"""

In [ ]:
system_recommender="""
You are a professional career development consultant. Based on the comparison between the job requirements and the candidate's CV, provide actionable recommendations for improvement:

1. **CV Enhancement Suggestions**:
   - Content additions (missing relevant experiences or skills to highlight)
   - Content modifications (how to better frame existing experience)
   - Format/structure improvements

2. **Skill Development Plan**:
   - Critical skills to acquire (prioritized by importance to the role)
   - Recommended learning resources (courses, certifications, projects)
   - Estimated timeline for skill acquisition

3. **Interview Preparation**:
   - How to address potential skill gaps during interviews
   - Key talking points to emphasize strengths relevant to this role

Provide practical, specific advice that would help this candidate become more competitive for this position.
"""

In [ ]:
def extract_text_from_pdf(pdf_file):
    if pdf_file is None:
        return "No file uploaded."

    text = ""
    try:
        # Open the PDF file
        doc = pymupdf .open(pdf_file)

        # Extract text from each page and join with page break character
        text = chr(12).join([page.get_text() for page in doc])

        # Close the document
        doc.close()

        return text
    except Exception as e:
        return f"Error extracting text: {str(e)}"
def get_feedback(job_desc, cv):
    cv=extract_text_from_pdf(cv)
    # Step 1:  Job Description Summary
    job_summarizer_prompt = f"Job Description：{job_desc}"
    job_summarizer = reply(system_job_summarizer, job_summarizer_prompt,
                          provider = provider_job_smmarizer,
                          model = model_job_smmarizer
                          )

    # Step 2: CV Summary
    cv_summarizer_prompt = f"cv content{cv}"
    cv_summarizer = reply(system_cv_summarizer, cv_summarizer_prompt,
                          provider = provider_cv_smmarizer,
                          model = model_cv_smmarizer
                          )

    # Step 3: Compare Job description and CV
    comparer_prompt = f"Job Summary:\n{job_summarizer} \nCV Summary:\n{cv_summarizer}"
    comparer = reply(system_comparer, comparer_prompt,
                          provider = provider_comparer,
                          model = model_comparer
                          )

    # Step 4: CV Improvement Recommender
    recommender_prompt =  f"Job Summary:\n{job_summarizer} \nCV Summary:\n{cv_summarizer}"
    recommender = reply(system_recommender, recommender_prompt,
                          provider = provider_recommender,
                          model = model_recommender
                          )

    return comparer, recommender

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr


In [ ]:
import gradio as gr
import pymupdf


with gr.Blocks() as demo:
    gr.Markdown("###  Job abd CV Compare System")
    gr.Markdown("Please Input the Job Description and Upload your CV for comparison")

    with gr.Row():
        job_desc = gr.Textbox(label="Job Description")
        pdf_upload = gr.File(label="Upload CV PDF", file_types=[".pdf"])

    btn = gr.Button("Check Job")

    with gr.Row():
        out1 = gr.Textbox(label="Job Match Summary")
        out2 = gr.Textbox(label="Improvement Recommendation")

    btn.click(get_feedback, inputs=[job_desc,pdf_upload], outputs=[out1, out2])


In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1f249f2171b14e4a6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1f249f2171b14e4a6e.gradio.live
